In [25]:
import numpy as np 
import pandas as pd

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

df = pd.read_csv('Train_ServicesOptedFor.csv')
df2 = pd.read_csv('Train_Demographics.csv')
df3 = pd.read_csv('Train (3).csv')

In [26]:
#Reformat Customer data by categories of service
customer_data = df.set_index(['CustomerID','TypeOfService']).unstack()
customer_data.columns = customer_data.columns.map('_'.join)
#Reindex for merge 
customer_data = customer_data.reset_index()
# Merge Dataframes together
merge_one = pd.merge(customer_data, df2, left_on='CustomerID', right_on='HouseholdID', how='outer').drop('HouseholdID', axis=1)
final = pd.merge(merge_one, df3, on='CustomerID', how='outer')

In [27]:
#Search for NAN values
final.HasDependents.isnull().values.any()

False

In [28]:
train_cats(final)

In [29]:
df, y, nas = proc_df(final, 'Churn')

In [30]:
final.columns

Index(['CustomerID', 'SeviceDetails_DeviceProtection',
       'SeviceDetails_HasPhoneService',
       'SeviceDetails_InternetServiceCategory', 'SeviceDetails_MultipleLines',
       'SeviceDetails_OnlineBackup', 'SeviceDetails_OnlineSecurity',
       'SeviceDetails_StreamingMovies', 'SeviceDetails_StreamingTelevision',
       'SeviceDetails_TechnicalSupport', 'Country', 'State', 'Retired',
       'HasPartner', 'HasDependents', 'Education', 'Gender', 'Churn'],
      dtype='object')

In [43]:
m =  RandomForestClassifier(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.98150245375613443

In [32]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 1000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(final, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((4298, 17), (4298,), (1000, 17))

In [33]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [47]:
m = RandomForestClassifier(n_estimators=80, min_samples_leaf=3, max_features=0.5,  n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.32465036496132377, 0.45166359162544856, 0.89460214053047926, 0.79600000000000004]
